# FLUX LoRA Training in Google Colab

이 노트북은 Google Colab에서 FLUX LoRA 훈련을 위한 환경설정 및 실행 가이드입니다.

In [ ]:
# GPU 정보 확인
!nvidia-smi
!nvcc --version

In [ ]:
# 최종 의존성 해결 방법 (모든 패키지 호환)
print("🎯 Final dependency resolution...")

# 1. 완전히 깔끔한 시작을 위한 주요 패키지 제거
!pip uninstall torch torchvision torchaudio numpy opencv-python opencv-python-headless -y

# 2. NumPy를 모든 패키지가 호환되는 버전으로 고정
# TensorFlow: <2.1.0, Numba: <2.1, OpenCV: >=2.0
!pip install "numpy>=2.0,<2.1" --force-reinstall

# 3. OpenCV 설치 (NumPy 2.0 호환)
!pip install opencv-python-headless>=4.9.0.80

# 4. PyTorch 설치 (모든 ML 패키지 의존성 만족)
!pip install torch>=2.0.0 torchvision>=0.15.0 torchaudio --index-url https://download.pytorch.org/whl/cu121

# 5. ML 패키지들 순차 설치
!pip install transformers>=4.21.0
!pip install diffusers>=0.21.0
!pip install peft>=0.4.0
!pip install accelerate>=0.21.0

# 6. 기타 필수 패키지
!pip install datasets Pillow pandas tqdm wandb safetensors
!pip install sentencepiece protobuf python-dotenv

# 7. 의존성 체크
print("🔍 Checking dependencies...")
!pip check

print("✅ Installation completed with compatible versions")
print("⚠️ Please restart runtime before proceeding")

In [ ]:
# 🚀 가장 간단한 해결책 (추천)
print("💡 Simplest solution: Use default Colab packages")

# Colab 기본 환경 그대로 사용하고 필수 패키지만 추가
# NumPy나 OpenCV 건드리지 않음

# PyTorch만 필요시 업데이트
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121 --upgrade

# FLUX 관련 필수 패키지만 설치
!pip install diffusers --upgrade
!pip install transformers --upgrade  
!pip install peft accelerate
!pip install sentencepiece protobuf python-dotenv

# 의존성 경고 무시하고 진행
print("✅ Essential packages installed")
print("⚠️ Dependency warnings can be safely ignored")
print("💡 FLUX training should work with this setup")

In [ ]:
# 대안: 깔끔한 환경으로 시작 (권장)
print("🔄 Alternative: Clean installation...")

# Colab 런타임을 완전히 리셋하는 것이 가장 확실함
print("💡 Most reliable method:")
print("1. Runtime → Disconnect and delete runtime")  
print("2. Runtime → Connect (new clean environment)")
print("3. Run this installation cell")

# 새 환경에서 올바른 순서로 설치
!pip install --upgrade pip setuptools wheel

# NumPy 2.x 계열로 통일 (모든 패키지가 요구하는 버전)
!pip install "numpy>=2.0,<2.3"

# PyTorch (최신 안정 버전)
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

# 핵심 ML 패키지들 (순서 중요)
!pip install transformers
!pip install diffusers 
!pip install peft accelerate

# 기타 필수 패키지들
!pip install datasets Pillow pandas tqdm wandb safetensors
!pip install sentencepiece protobuf python-dotenv

print("✅ Clean installation completed")

In [ ]:
# 설치 검증 (런타임 재시작 후 실행)
print("🔍 Verifying installation...")

try:
    import numpy as np
    print(f"✅ NumPy version: {np.__version__}")
    
    import torch
    print(f"✅ PyTorch version: {torch.__version__}")
    print(f"✅ CUDA available: {torch.cuda.is_available()}")
    
    if torch.cuda.is_available():
        print(f"✅ CUDA version: {torch.version.cuda}")
        print(f"✅ GPU: {torch.cuda.get_device_name()}")
        print(f"✅ VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f}GB")
    
    from diffusers import FluxPipeline
    print("✅ Diffusers imported successfully")
    
    import transformers
    print(f"✅ Transformers version: {transformers.__version__}")
    
    import peft
    print(f"✅ PEFT version: {peft.__version__}")
    
    print("\\n🎉 All packages installed correctly!")
    print("💡 You can now proceed with the training")
    
except ImportError as e:
    print(f"❌ Import error: {e}")
    print("💡 Please check package installation")
except Exception as e:
    print(f"❌ Error: {e}")
    print("💡 Please restart runtime and try again")

In [ ]:
# 환경변수 및 GitHub 토큰 설정
import os
from google.colab import userdata

# 방법 1: Colab Secrets 사용 (권장)
# 🔑 키 아이콘 클릭 → Add new secret:
# HUGGINGFACE_TOKEN: hf_your_token_here
# GITHUB_TOKEN: ghp_your_token_here (Private 저장소용)

try:
    HF_TOKEN = userdata.get('HUGGINGFACE_TOKEN')
    GITHUB_TOKEN = userdata.get('GITHUB_TOKEN')
    
    os.environ['HUGGINGFACE_TOKEN'] = HF_TOKEN
    os.environ['GITHUB_TOKEN'] = GITHUB_TOKEN
    
    print("✅ Tokens loaded from Colab secrets")
except:
    # 방법 2: 직접 입력 (임시용)
    HF_TOKEN = "hf_your_token_here"
    GITHUB_TOKEN = "ghp_your_token_here"
    
    os.environ['HUGGINGFACE_TOKEN'] = HF_TOKEN
    os.environ['GITHUB_TOKEN'] = GITHUB_TOKEN
    
    print("⚠️ Tokens set manually")

# 기타 환경변수 설정
os.environ['FLUX_MODEL_NAME'] = 'black-forest-labs/FLUX.1-schnell'
os.environ['HF_HUB_DISABLE_SYMLINKS_WARNING'] = '1'
print("✅ Environment variables configured")

In [ ]:
# Private GitHub 저장소 클로닝
import os

# GitHub 토큰 확인
github_token = os.environ.get('GITHUB_TOKEN')

if github_token and github_token != "ghp_your_token_here":
    # 토큰을 사용한 Private 저장소 클로닝
    !git clone https://{github_token}@github.com/comking2/flux-lora-training.git
    print("✅ Private repository cloned successfully")
else:
    # 토큰이 없으면 Public 저장소 시도 또는 수동 업로드 안내
    try:
        !git clone https://github.com/comking2/flux-lora-training.git
        print("✅ Public repository cloned")
    except:
        print("❌ Repository access failed")
        print("💡 해결방법:")
        print("1. GitHub Token을 Colab Secrets에 추가")
        print("2. 또는 저장소를 Public으로 변경")
        print("3. 또는 파일을 직접 업로드")

# 클로닝된 디렉토리로 이동
try:
    %cd flux-lora-training
    print("📁 Moved to project directory")
    
    # 파일 목록 확인
    !ls -la
except:
    print("⚠️ Directory not found - please check cloning status")

In [ ]:
# 대안: 개별 파일 다운로드 (클로닝 실패 시)
# GitHub Token이 없거나 Private 저장소 접근이 안 될 때 사용

print("🔄 Alternative: Downloading individual files...")

# 주요 파일들을 개별적으로 다운로드
files_to_download = [
    'train_lora.py',
    'dataset_processor.py', 
    'inference.py',
    'config.json',
    'requirements.txt',
    '.env.example'
]

import requests
import os

def download_file(filename, token=None):
    if token:
        url = f"https://api.github.com/repos/comking2/flux-lora-training/contents/{filename}"
        headers = {'Authorization': f'token {token}'}
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            import base64
            content = base64.b64decode(response.json()['content']).decode('utf-8')
            with open(filename, 'w') as f:
                f.write(content)
            return True
    return False

# GitHub API를 통한 다운로드 시도
github_token = os.environ.get('GITHUB_TOKEN')
downloaded = []

for filename in files_to_download:
    if download_file(filename, github_token):
        downloaded.append(filename)
        print(f"✅ Downloaded: {filename}")
    else:
        print(f"❌ Failed: {filename}")

if downloaded:
    print(f"\\n✅ Successfully downloaded {len(downloaded)} files")
    !ls -la
else:
    print("\\n💡 Please upload files manually or provide GitHub token")

In [ ]:
# 데이터셋 업로드 또는 다운로드
# 방법 1: Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

# 방법 2: 직접 업로드
# from google.colab import files
# uploaded = files.upload()

In [ ]:
# GPU 메모리 최적화 설정
import torch

# GPU 메모리 확인
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name()}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f}GB")
    
    # 메모리 캐시 정리
    torch.cuda.empty_cache()

In [ ]:
# FLUX LoRA 훈련 실행
# T4 (16GB) 환경 최적화

!python train_lora.py \
    --data_dir "/content/drive/MyDrive/training_data" \
    --output_dir "./flux_lora_output" \
    --epochs 10 \
    --batch_size 1 \
    --lora_rank 8 \
    --learning_rate 5e-5